In [1]:
import os
import json
import pandas as pd

# EDA

In [3]:
with open('annotations.json') as json_file: 
    data = json.load(json_file) 

In [4]:
data.keys()

dict_keys(['info', 'images', 'annotations', 'scene_annotations', 'licenses', 'categories', 'scene_categories'])

In [5]:
data['info'].keys()

dict_keys(['year', 'version', 'description', 'contributor', 'url', 'date_created'])

In [6]:
len(data['images'])

1500

In [7]:
pd.DataFrame(data['images']).head()

,id,width,height,file_name,license,flickr_url,coco_url,date_captured,flickr_640_url
0,0,1537,2049,batch_1/000006.jpg,None,https://farm66.staticflickr.com/65535/33978196...,None,None,https://farm66.staticflickr.com/65535/33978196...
1,1,1537,2049,batch_1/000008.jpg,None,https://farm66.staticflickr.com/65535/47803331...,None,None,https://farm66.staticflickr.com/65535/47803331...
2,2,1537,2049,batch_1/000010.jpg,None,https://farm66.staticflickr.com/65535/40888872...,None,None,https://farm66.staticflickr.com/65535/40888872...
3,3,2049,1537,batch_1/000019.jpg,None,https://farm66.staticflickr.com/65535/47803331...,None,None,https://farm66.staticflickr.com/65535/47803331...
4,4,1537,2049,batch_1/000026.jpg,None,https://farm66.staticflickr.com/65535/33978199...,None,None,https://farm66.staticflickr.com/65535/33978199...


In [8]:
pd.DataFrame(data['annotations']).head()

,id,image_id,category_id,segmentation,area,bbox,iscrowd
0,1,0,6,"[[561.0, 1238.0, 568.0, 1201.0, 567.0, 1175.0,...",403954.0,"[517.0, 127.0, 447.0, 1322.0]",0
1,2,1,18,"[[928.0, 1876.0, 938.0, 1856.0, 968.0, 1826.0,...",1071259.5,"[1.0, 457.0, 1429.0, 1519.0]",0
2,3,1,14,"[[617.0, 383.0, 703.0, 437.0, 713.0, 456.0, 72...",99583.5,"[531.0, 292.0, 1006.0, 672.0]",0
3,4,2,5,"[[670.0, 993.0, 679.0, 998.0, 684.0, 1001.0, 6...",73832.5,"[632.0, 987.0, 500.0, 374.0]",0
4,5,2,7,"[[647.0, 1028.0, 650.0, 1022.0, 653.0, 1016.0,...",915.0,"[632.0, 989.0, 44.0, 51.0]",0


# Functions
get_img_shape - Reads the image from path
convert_labels - Definition: Parses label files to extract label and bounding box coordinates. Converts (x1, y1, x1, y2) KITTI format to (x, y, width, height) normalized YOLO format.

In [143]:
import cv2



def get_img_shape(path):
    """
    Reads the image from path
    """
    img = cv2.imread(path)
    try:
        return img.shape
    except AttributeError:
        print('error! ', path)
        return (None, None, None)
    
def convert_labels(path, x1, y1, x2, y2):
    '''
    Definition: Parses label files to extract label and bounding box
    coordinates. Converts (x1, y1, x1, y2) KITTI format to
    (x, y, width, height) normalized YOLO format.
    '''
    def sorting(l1, l2):
        if l1 > l2:
            lmax, lmin = l1, l2
            return lmax, lmin
        else:
            lmax, lmin = l2, l1
            return lmax, lmin
    size = get_img_shape(path)
    xmax, xmin = sorting(x1, x2)
    ymax, ymin = sorting(y1, y2)
    print(size[1])
    dw = (1.0)/size[1]
    dh = (1.0)/size[0]
    x = (xmin + xmax)/2.0
    y = (ymin + ymax)/2.0
    w = xmax - xmin
    h = ymax - ymin
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

# Script to run

In [216]:
# Reads the annotations file (JSON) - This in COCO Format
with open('annotations.json') as json_file: 
    training_data = json.load(json_file) 

#Cleaning Method - Converts the JSON script into dataframes and does some initial cleaning
df_images = pd.DataFrame(training_data['images'])
categories = pd.DataFrame(training_data['categories'])
df_annotations = pd.DataFrame(training_data['annotations'])
categories['name']=categories['name'].apply(lambda x: '_'.join(x.lower().split(' ')))
categories['supercategory']=categories['supercategory'].apply(lambda x: '_'.join(x.lower().split(' ')))

#--- RUN ONCE--- This is to create each folder with the object name
#for x in folders:
#    os.mkdir('final/'+x)

check_set = set() 
# ImageID = represents the images in total (about 1500)
# Annotations = represents a box within the images. So one image can have multiple annotations - I believe the TACO DATASET has 4000+
# So we created a set variable to ensure we don't create NEW images if it already exists 

for i in range(len(training_data['annotations'])):
    # Loops through each annotations
    if i%50==0:
        print(i)
        
    # Collects ID/annotation box = bbox, image location, category name (what annotation does this fall into?), filename
    # shutil.copy2 - copies the file from existing directory and moves it a folder
    
    image_id = training_data['annotations'][i]['image_id']
    category_id = str(training_data['annotations'][i]['category_id'])
    bbox = training_data['annotations'][i]['bbox']
    image_path = (df_images[df_images['id']==image_id])['file_name'].values[0]
    category_name = categories[categories['id']==int(category_id)]['name'].values[0]
    kitti_bbox = [bbox[0], bbox[1], bbox[2] + bbox[0], bbox[3] + bbox[1]]
    yolo_bbox = convert_labels(image_path, kitti_bbox[0], kitti_bbox[1], kitti_bbox[2], kitti_bbox[3])
    filename = 'final/'+category_name+'/'+category_name+str(image_id) + ".txt"
    content = str(int(category_id)+100) + " " + str(yolo_bbox[0]) + " " + str(yolo_bbox[1]) + " " + str(yolo_bbox[2]) + " " + str(yolo_bbox[3])
    shutil.copy2(image_path, ('final/'+category_name+'/'+category_name+str(image_id) +'.jpg')) # complete target filename given
    if image_id in check_set:
        # Append to file files
        file = open(filename, "a")
        file.write(content)
        file.write("\n")
        file.close()
    else:
        check_set.add(image_id)
        # Write files
        file = open(filename, "w")
        file.write(content)
        file.write("\n")
        file.close()

0
1537
1537
1537
1537
1537
2049
2049
2049
2049
1537
2049
2049
2049
2049
2049
2049
1537
2049
2049
1537
1537
1537
1537
1537
1537
1537
1537
1537
1537
1537
1537
2049
2049
2049
2049
2049
2049
2049
2049
2049
2049
1537
1537
1537
1537
1537
1537
1537
1537
1537
50
1537
1537
1537
1537
1537
2049
2049
2049
2049
2049
2049
1537
1537
1537
1537
1537
1537
1537
2049
2049
2049
2049
1537
1537
1537
1537
1537
1537
1537
1537
1537
1537
1537
1537
1537
1537
2049
2049
2049
2049
1537
1537
1537
1537
1537
1537
1537
1537
1537
1537
100
1537
1537
1537
1537
1537
1537
1537
1537
1537
2049
2049
2049
2049
2049
2049
2049
2049
2049
2049
2049
2049
2049
2049
2049
2049
2049
3264
3264
3264
3264
3264
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
150
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
3264
3264
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
24

4160
4160
4160
4160
4160
3120
3120
3120
3120
3120
3120
3120
3120
3120
3120
4160
3024
3024
3024
3024
3024
3024
3024
3024
3024
3024
3024
4160
3120
3120
3120
4160
3120
4160
4160
3120
4160
3120
3120
1650
3120
4160
4160
4160
3120
3120
3120
3120
3120
3120
3120
3120
3120
4160
4160
4160
4160
4160
4160
4160
4160
4160
4160
4160
4160
4160
3120
3120
3120
3120
3120
4160
3120
3120
4160
4160
3120
3120
3120
4160
4160
4160
3120
3120
3120
4160
4160
4160
4160
4160
1700
4160
4160
4160
4160
4160
4160
4160
3120
3120
3120
4160
4160
3120
3120
3120
4160
4160
4160
4160
4160
3120
4160
4160
4160
4160
4160
4160
3120
3120
4160
3024
3024
3024
3024
3456
3456
3456
3456
3456
3120
3120
3120
3120
3120
3024
3024
3024
3024
3024
3024
1750
4032
4032
3024
3024
3024
3024
3024
3024
3024
3024
3024
3024
3024
3024
3024
3024
4032
4032
4032
3024
3024
3024
4032
3024
3024
3024
3024
842
842
3024
3024
4032
3024
3024
4032
4032
3024
3024
4032
4032
3024
3024
3024
3024
3024
3024
3024
3024
3024
3024
1800
3024
3024
3024
3024
3024
3024
3024
30

2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
3264
3264
3264
3264
3264
2448
2448
2448
2448
2448
2624
2624
3264
3264
2624
2624
2624
2624
2624
3250
2448
3680
3680
3680
3680
3680
3680
3680
2448
2624
2448
3264
3264
2448
2448
2448
2448
3264
2448
3644
2624
2624
2624
2624
2448
2867
2867
3680
3680
3264
3264
2448
2624
2624
2624
2624
2624
3680
3680
2275
2275
2275
2448
3264
3264
3264
3264
3264
2448
2448
3300
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
3350
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2448
2624
3264
3264
3264
3264
3264
3264
3264
3264
3264
3264
3400
3264
3264
2448
2448
2448
2448
2624
2624
2624
2624
2624
3264
3264
2448
2448
2448
